In [1]:
import pandas as pd
import geopandas as gpd
import os
import boto3, json
import datetime as dt
import plotly.express as px
import statsmodels
import shapely

#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

# SET UP DATA

- Set up Tennessee Census Tract Data: Read from the shapefile and select its GEOID, wkt, and COUNTYFP columns to CSV to be uploaded to Athena. Note that the incidents data (nfd_incidents_xd_seg.parquet), the economic data (census_economic_data_all_tennessee.csv), and the weather data are all uploaded to Athena directly through the AWS console.

In [ ]:
df_tracts = gpd.read_file('tn_census/tl_2016_47_tract.shp')
df_tracts['wkt'] = pd.Series(
        map(lambda geom: str(geom.wkt), df_tracts['geometry']),
        index=df_tracts.index, dtype='string')
df_tracts_edited = df_tracts[['GEOID', 'wkt', 'COUNTYFP']]
df_tracts_edited.to_csv('df_tracts.csv', index=False)

- Set up Athena credentials and connection

In [ ]:
credentials = {
    'region_name': 'us-east-1',
    'aws_access_key_id': 'xxx',
    'aws_secret_access_key': 'xxx',
    'aws_session_token': 'xxx'
}

session = boto3.session.Session(**credentials)
athena = session.client('athena', region_name='us-east-1')
s3 = session.client('s3')

## SET UP CENSUS/INCIDENT TABLES

- **Create econ_tracts table:** Join Davidson County tracts (COUNTYFP=037) with their economic data by GEOID, and store as a table in Athena

In [ ]:
athena.start_query_execution(
    QueryString = 'CREATE TABLE "econ_tracts" AS (SELECT * FROM ((SELECT * FROM "tracts" WHERE countyfp = \'037\' AND geoid IS NOT NULL AND wkt IS NOT NULL) AS "tracts1") LEFT JOIN ((SELECT *, CONCAT(CAST(state AS varchar(2)), LPAD(CAST(county AS varchar(3)), 3, \'0\'), LPAD(CAST(tract AS varchar(6)), 6, \'0\')) AS GEO_ID FROM "econ") AS "econ1") ON "tracts1".GEOID = "econ1".GEO_ID);',
    QueryExecutionContext = {
        'Database': 'econ'
    },
    ResultConfiguration={
        'OutputLocation': 's3://final-proj-schachtj/output'
    }
)

{'QueryExecutionId': '1f6f88e9-881b-42a7-83e2-9dce8ecc516e',
 'ResponseMetadata': {'RequestId': 'c74b5f51-ed91-45cf-9c4c-80a263ccae34',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 25 Apr 2023 03:51:47 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '59',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'c74b5f51-ed91-45cf-9c4c-80a263ccae34'},
  'RetryAttempts': 0}}

In [ ]:
athena.get_query_execution(QueryExecutionId = '1f6f88e9-881b-42a7-83e2-9dce8ecc516e')

{'QueryExecution': {'QueryExecutionId': '1f6f88e9-881b-42a7-83e2-9dce8ecc516e',
  'Query': 'CREATE TABLE "econ_tracts" AS (SELECT * FROM ((SELECT * FROM "tracts" WHERE countyfp = \'037\' AND geoid IS NOT NULL AND wkt IS NOT NULL) AS "tracts1") LEFT JOIN ((SELECT *, CONCAT(CAST(state AS varchar(2)), LPAD(CAST(county AS varchar(3)), 3, \'0\'), LPAD(CAST(tract AS varchar(6)), 6, \'0\')) AS GEO_ID FROM "econ") AS "econ1") ON "tracts1".GEOID = "econ1".GEO_ID)',
  'StatementType': 'DDL',
  'ResultConfiguration': {'OutputLocation': 's3://final-proj-schachtj/output/tables/1f6f88e9-881b-42a7-83e2-9dce8ecc516e'},
  'ResultReuseConfiguration': {'ResultReuseByAgeConfiguration': {'Enabled': False}},
  'QueryExecutionContext': {'Database': 'econ'},
  'Status': {'State': 'SUCCEEDED',
   'SubmissionDateTime': datetime.datetime(2023, 4, 24, 22, 51, 47, 148000, tzinfo=tzlocal()),
   'CompletionDateTime': datetime.datetime(2023, 4, 24, 22, 51, 49, 552000, tzinfo=tzlocal())},
  'Statistics': {'EngineExecu

- **Create econ_incidents table**: Join incidents with the demographic information of the tract they take place in

In [ ]:
athena.start_query_execution(
    QueryString = 'CREATE TABLE "econ_incidents" AS (SELECT incident_id, median_household_income, median_family_income, median_housing_value, total_population, white, african_american, median_gross_rent, hispanic, MONTH(time_local) AS incident_month, response_time_sec, day_of_week, weekend_or_not, geometry, countyfp, geoid, wkt FROM ((SELECT geoid, wkt, median_household_income, median_family_income, median_housing_value, total_population, white, african_american, median_gross_rent, hispanic, countyfp FROM "econ_tracts") AS "econtracts1") RIGHT JOIN ((SELECT response_time_sec, day_of_week, time_local, weekend_or_not, geometry, incident_id FROM "incidents") AS "incidents1") ON ST_CONTAINS (ST_GeometryFromText("econtracts1".wkt), ST_GeometryFromText("incidents1".geometry)) WHERE countyfp IS NOT NULL);',
    QueryExecutionContext = {
        'Database': 'econ'
    },
    ResultConfiguration={
        'OutputLocation': 's3://final-proj-schachtj/output'
    }
)

{'QueryExecutionId': 'd549ea68-e6c9-438a-b38f-0dbb32097379',
 'ResponseMetadata': {'RequestId': 'abf25a41-ace8-4f24-b19d-5b5208acd8e1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 26 Apr 2023 22:08:42 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '59',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'abf25a41-ace8-4f24-b19d-5b5208acd8e1'},
  'RetryAttempts': 0}}

In [ ]:
athena.get_query_execution(QueryExecutionId = 'd549ea68-e6c9-438a-b38f-0dbb32097379')

{'QueryExecution': {'QueryExecutionId': 'd549ea68-e6c9-438a-b38f-0dbb32097379',
  'Query': 'CREATE TABLE "econ_incidents" AS (SELECT incident_id, median_household_income, median_family_income, median_housing_value, total_population, white, african_american, median_gross_rent, hispanic, MONTH(time_local) AS incident_month, response_time_sec, day_of_week, weekend_or_not, geometry, countyfp, geoid, wkt FROM ((SELECT geoid, wkt, median_household_income, median_family_income, median_housing_value, total_population, white, african_american, median_gross_rent, hispanic, countyfp FROM "econ_tracts") AS "econtracts1") RIGHT JOIN ((SELECT response_time_sec, day_of_week, time_local, weekend_or_not, geometry, incident_id FROM "incidents") AS "incidents1") ON ST_CONTAINS (ST_GeometryFromText("econtracts1".wkt), ST_GeometryFromText("incidents1".geometry)) WHERE countyfp IS NOT NULL)',
  'StatementType': 'DDL',
  'ResultConfiguration': {'OutputLocation': 's3://final-proj-schachtj/output/tables/d549ea

## 1. FREQUENCY OF INCIDENTS BY CENSUS TRACT ACROSS NASHVILLE

- Get number of incidents for each tract over the time period of 2017-2021

In [ ]:
athena.start_query_execution(
    QueryString = 'SELECT geoid, wkt, COUNT(*) AS num_incidents, AVG(response_time_sec) AS avg_response_time, VARIANCE(response_time_sec) AS var_response_time, AVG(total_population) AS total_population, COUNT(*)/AVG(total_population) AS incidents_per_capita FROM "econ_incidents" GROUP BY geoid, wkt;',
    QueryExecutionContext = {
        'Database': 'econ'
    },
    ResultConfiguration={
        'OutputLocation': 's3://final-proj-schachtj/output'
    }
)

{'QueryExecutionId': '32742741-7ace-4f65-9a1d-1653ec065a6a',
 'ResponseMetadata': {'RequestId': 'd13108d9-27bf-4c1e-a68b-bdd06e89bfac',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 27 Apr 2023 04:12:02 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '59',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'd13108d9-27bf-4c1e-a68b-bdd06e89bfac'},
  'RetryAttempts': 0}}

In [ ]:
athena.get_query_execution(QueryExecutionId = '32742741-7ace-4f65-9a1d-1653ec065a6a')

{'QueryExecution': {'QueryExecutionId': '32742741-7ace-4f65-9a1d-1653ec065a6a',
  'Query': 'SELECT geoid, wkt, COUNT(*) AS num_incidents, AVG(response_time_sec) AS avg_response_time, VARIANCE(response_time_sec) AS var_response_time, AVG(total_population) AS total_population, COUNT(*)/AVG(total_population) AS incidents_per_capita FROM "econ_incidents" GROUP BY geoid, wkt',
  'StatementType': 'DML',
  'ResultConfiguration': {'OutputLocation': 's3://final-proj-schachtj/output/32742741-7ace-4f65-9a1d-1653ec065a6a.csv'},
  'ResultReuseConfiguration': {'ResultReuseByAgeConfiguration': {'Enabled': False}},
  'QueryExecutionContext': {'Database': 'econ'},
  'Status': {'State': 'SUCCEEDED',
   'SubmissionDateTime': datetime.datetime(2023, 4, 26, 23, 12, 2, 464000, tzinfo=tzlocal()),
   'CompletionDateTime': datetime.datetime(2023, 4, 26, 23, 12, 3, 592000, tzinfo=tzlocal())},
  'Statistics': {'EngineExecutionTimeInMillis': 868,
   'DataScannedInBytes': 1050718,
   'TotalExecutionTimeInMillis': 

In [ ]:
s3.download_file('final-proj-schachtj', 'output/32742741-7ace-4f65-9a1d-1653ec065a6a.csv', 'incidents_by_tract.csv')

- Read into CSV, create geometry column, and make GeoPandas dataframe

In [3]:
incidents_by_tract = pd.read_csv('incidents_by_tract.csv')
incidents_by_tract['geometry'] = incidents_by_tract['wkt'].apply(shapely.wkt.loads)
incidents_by_tract = incidents_by_tract.drop('wkt', axis=1)
incidents_by_tract

,geoid,num_incidents,avg_response_time,var_response_time,total_population,incidents_per_capita,geometry
0,47037010301,62,204.101695,35274.748101,3739.0,0.016582,"POLYGON ((-86.74131799999999 36.293537, -86.74..."
1,47037010103,227,630.943925,80495.978062,2627.0,0.086410,"POLYGON ((-86.917518 36.339757, -86.9174659999..."
2,47037015612,407,405.320413,64346.503287,6690.0,0.060837,"POLYGON ((-86.666478 36.103628, -86.666206 36...."
3,47037010904,248,399.751073,52001.342941,2761.0,0.089823,"POLYGON ((-86.78634699999999 36.237884, -86.78..."
4,47037011100,86,218.548780,48659.806233,4344.0,0.019797,"POLYGON ((-86.73436 36.221584, -86.73425999999..."
...,...,...,...,...,...,...,...
156,47037018405,74,498.041667,157743.899648,4710.0,0.015711,"POLYGON ((-86.96850099999999 36.027814, -86.96..."
157,47037018201,135,437.092308,70099.572809,3069.0,0.043988,POLYGON ((-86.90934299999999 36.11469599999999...
158,47037015614,121,421.601695,68634.669057,4572.0,0.026465,"POLYGON ((-86.67002 36.078603, -86.669805 36.0..."
159,47037015803,278,404.544061,80373.025936,2622.0,0.106026,"POLYGON ((-86.72737 36.13836, -86.727336 36.13..."


In [4]:
inc_tract_gdf = gpd.GeoDataFrame(incidents_by_tract, crs='epsg:4326')
inc_tract_gdf.index = inc_tract_gdf['geoid']
inc_tract_gdf.index

Int64Index([47037010301, 47037010103, 47037015612, 47037010904, 47037011100,
            47037018202, 47037019006, 47037015620, 47037011600, 47037012600,
            ...
            47037012200, 47037010701, 47037010302, 47037011400, 47037019110,
            47037018405, 47037018201, 47037015614, 47037015803, 47037013201],
           dtype='int64', name='geoid', length=161)

- Plot the relationship between population for each tract and number of incidents

In [6]:
fig = px.scatter(inc_tract_gdf, x='total_population', y='num_incidents', trendline='ols',
    title="Number of Incidents by Total Population for Census Tracts, Davidson County 2017-2021")
fig.show()
fig.write_html('num_incidents_by_total_pop.html')

- Plot figures with each tract colored by the number of incidents. Those with the most incidents are the most yellow.

In [7]:
fig = px.choropleth(inc_tract_gdf, geojson=inc_tract_gdf.geometry, locations = inc_tract_gdf.index,
                            color='num_incidents',
                            color_continuous_scale="Viridis",
                            range_color=(0, inc_tract_gdf['num_incidents'].max()),
                            scope = "usa",
                            title="Number of Incidents by Census Tract, Davidson County 2017-2021")
fig.update_geos(fitbounds="locations", visible=True)
fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig.write_html('incidents_by_census_tract_choropleth.html')

- Plot figure with each tract colored by the number of incidents per capita. Those with the most incidents per capita are the most yellow. Two tracts have been lost in the plot, likely since the population data was not available for those tracts.

In [8]:
fig = px.choropleth(inc_tract_gdf, geojson=inc_tract_gdf.geometry, locations = inc_tract_gdf.index,
                            color='incidents_per_capita',
                            color_continuous_scale="Viridis",
                            range_color=(0, inc_tract_gdf['incidents_per_capita'].max()),
                            scope = "usa",
                            title="Number of Incidents PER CAPITA by Census Tract, Davidson County 2017-2021")
fig.update_geos(fitbounds="locations", visible=True)
fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig.write_html('incidents_per_capita_by_census_tract_choropleth.html')

- **Notable Conclusions for Section 1: Frequency of Incidents by Census Tract** 
    - An increased tract population in Davidson County does not seem to significantly increase the number of incidents.
    - Both total incidents and incidents per capita were highest in the center of the city, and incidents per capita seemed to increase steadily as the regions moved from the outer regions of Davidson County towards the center. This indicates that there should be more emergency response manpower located near the center of the city. 

## 2: TEMPORAL ANALYSIS OF ACCIDENTS BY MONTH ACROSS NASHVILLE

- Get number of incidents by month. Properly account for the possibility that some values factored into averages can be null or less than zero.

In [ ]:
athena.start_query_execution(
    QueryString = 'SELECT incident_month, COUNT(*) AS num_incidents, AVG(CASE WHEN median_household_income > 0 THEN median_household_income ELSE NULL END) AS median_household_income, AVG(CASE WHEN median_family_income > 0 THEN median_family_income ELSE NULL END) AS median_family_income, AVG(CASE WHEN median_housing_value > 0 THEN median_housing_value ELSE NULL END) AS median_housing_value, AVG(CASE WHEN median_gross_rent > 0 THEN median_gross_rent ELSE NULL END) AS median_gross_rent, AVG(CASE WHEN total_population > 0 THEN total_population ELSE NULL END) AS total_population, AVG(CASE WHEN white > 0 THEN white ELSE NULL END) AS white, AVG(CASE WHEN african_american > 0 THEN african_american ELSE NULL END) AS african_american, AVG(CASE WHEN hispanic > 0 THEN hispanic ELSE NULL END) AS hispanic, AVG(CASE WHEN response_time_sec > 0 THEN response_time_sec ELSE NULL END) AS avg_response_time FROM "econ_incidents" GROUP BY incident_month;',
    QueryExecutionContext = {
        'Database': 'econ'
    },
    ResultConfiguration={
        'OutputLocation': 's3://final-proj-schachtj/output'
    }
)

{'QueryExecutionId': 'c5345efb-be70-4f5d-a978-bb9ee50db79e',
 'ResponseMetadata': {'RequestId': 'f9ad8960-a1aa-4481-8782-8e925ce2b51c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 26 Apr 2023 22:13:40 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '59',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'f9ad8960-a1aa-4481-8782-8e925ce2b51c'},
  'RetryAttempts': 0}}

In [ ]:
athena.get_query_execution(QueryExecutionId = 'c5345efb-be70-4f5d-a978-bb9ee50db79e')

{'QueryExecution': {'QueryExecutionId': 'c5345efb-be70-4f5d-a978-bb9ee50db79e',
  'Query': 'SELECT incident_month, COUNT(*) AS num_incidents, AVG(CASE WHEN median_household_income > 0 THEN median_household_income ELSE NULL END) AS median_household_income, AVG(CASE WHEN median_family_income > 0 THEN median_family_income ELSE NULL END) AS median_family_income, AVG(CASE WHEN median_housing_value > 0 THEN median_housing_value ELSE NULL END) AS median_housing_value, AVG(CASE WHEN median_gross_rent > 0 THEN median_gross_rent ELSE NULL END) AS median_gross_rent, AVG(CASE WHEN total_population > 0 THEN total_population ELSE NULL END) AS total_population, AVG(CASE WHEN white > 0 THEN white ELSE NULL END) AS white, AVG(CASE WHEN african_american > 0 THEN african_american ELSE NULL END) AS african_american, AVG(CASE WHEN hispanic > 0 THEN hispanic ELSE NULL END) AS hispanic, AVG(CASE WHEN response_time_sec > 0 THEN response_time_sec ELSE NULL END) AS avg_response_time FROM "econ_incidents" GROUP 

In [ ]:
s3.download_file('final-proj-schachtj', 'output/c5345efb-be70-4f5d-a978-bb9ee50db79e.csv', 'incidents_by_month.csv')

In [9]:
incidents_by_month = pd.read_csv('incidents_by_month.csv')
incidents_by_month

,incident_month,num_incidents,median_household_income,median_family_income,median_housing_value,median_gross_rent,total_population,white,african_american,hispanic,avg_response_time
0,10,2653,47561.461420,59234.912558,187657.408844,931.154083,4548.271822,2649.527182,1470.209186,516.528526,416.059608
1,6,2264,48464.018132,59392.154649,189521.675774,918.232601,4539.787523,2646.148734,1473.897810,518.098678,394.724380
2,11,2455,48860.966611,60237.514023,192259.873950,928.895334,4513.693139,2667.218711,1427.315789,519.982624,416.170852
3,1,2996,49150.043806,60590.368782,194342.249135,937.913704,4605.079918,2705.536885,1472.255540,519.096476,398.275862
4,8,2537,47423.152817,58925.665855,189056.186198,918.802857,4525.168353,2648.579927,1447.378930,523.248943,402.101487
5,4,2306,48994.184725,59994.517595,191982.184423,928.244852,4573.880762,2659.424202,1501.785425,502.283141,379.596988
6,12,2247,48538.062557,59497.030819,188527.713626,927.781439,4612.017915,2691.892513,1485.460502,536.508864,411.549289
7,5,2411,48583.825478,59621.625000,190802.873070,931.198379,4640.206692,2703.308344,1513.222366,518.357677,370.218466
8,3,2383,48016.955584,59415.726918,189198.609300,918.516536,4486.978513,2622.410400,1446.991757,515.072225,392.772606
9,7,2297,47780.628469,58561.307796,187441.494149,921.456101,4610.318141,2653.467382,1514.796296,532.168216,385.418873


In [10]:
fig = px.bar(incidents_by_month, x='incident_month', y='num_incidents', title='Number of Incidents by Month, Davidson County 2017-2021')
fig.show()
fig.write_html('incidents_by_month.html')

In [ ]:
janfeb = incidents_by_month[(incidents_by_month['incident_month']<=2)]
janfeb_freq = sum(janfeb['num_incidents'])/len(janfeb)
allothers = incidents_by_month[incidents_by_month['incident_month']>2]
allothers_freq = sum(allothers['num_incidents'])/len(allothers)
print('The accidents per month in January/Feburary is ' + str(janfeb_freq) + ' whereas the accidents per month across all other months is ' + str(allothers_freq) + ', which is an increase of ' + str(((janfeb_freq/allothers_freq)-1)*100) + ' percent')

The accidents per month in January/Feburary is 2901.5 whereas the accidents per month across all other months is 2381.5, which is an increase of 21.834977955070343 percent


In [11]:
fig = px.bar(incidents_by_month, x='incident_month', y='avg_response_time', title='Average Response Time on Incidents by Month, Davidson County 2017-2021')
fig.show()
fig.write_html('avg_res_time_by_month.html')

- **Notable Conclusions for Section 2: Temporal Analysis of Accidents By Month**
    - By month, the average response time and the average tract household income across incidents do not appear to undergo any significant change.
    - Incidents seem to occur most often during the first few months of the year, with >21% more acccidents per month compared to the rest of the year. This is likely due to the cold temperatures and snow that are more prevalent during January and February. This also indicates that there should be more emergency response manpower during those first two months. 

## 3: RESPONSE TIME DISTRIBUTION ON INCIDENTS ACROSS CENSUS TRACTS

- Take the incidents_by_tract dataframe from Section 1, and make plots regarding response time

In [12]:
fig = px.scatter(inc_tract_gdf, x='total_population', y='avg_response_time', trendline='ols',
    title="Average Response Time by Tract Population for Census Tracts, Davidson County 2017-2021")
fig.show()
fig.write_html('avg_res_by_tract_population.html')

In [14]:
fig = px.choropleth(inc_tract_gdf, geojson=inc_tract_gdf.geometry, locations = inc_tract_gdf.index,
                            color='avg_response_time',
                            color_continuous_scale="Viridis",
                            range_color=(inc_tract_gdf['avg_response_time'].min(), inc_tract_gdf['avg_response_time'].max()),
                            scope = "usa",
                            title="Average Response Time by Census Tract, Davidson County 2017-2021")
fig.update_geos(fitbounds="locations", visible=True)
fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig.write_html('avg_res_time_by_tract.html')

In [15]:
fig = px.choropleth(inc_tract_gdf, geojson=inc_tract_gdf.geometry, locations = inc_tract_gdf.index,
                            color='total_population',
                            color_continuous_scale="Viridis",
                            range_color=(inc_tract_gdf['total_population'].min(), inc_tract_gdf['total_population'].max()),
                            scope = "usa",
                            title="Population by Census Tract, Davidson County 2017-2021")
fig.update_geos(fitbounds="locations", visible=True)
fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig.write_html('pop_by_tract.html')

- **Notable Conclusions for Section 3: Response Time Distribution on Incidents Across Census Tracts**
    - The outer regions of Davidson County that are further away from the city tend to have the highest response times. This implies that emergency response locations in areas farther from the city are more spread out or that those regions have a perceived lower priority. This indicates that there should be a greater emergency response presence in those outer regions, especially on the west side of Davidson County.
    - There is a slightly positive correlation between the number of people in a tract and its average response time. A few of the regions with the highest populations also have high response times, notably tracts 47037019114 (southeast corner of Davidson) and 47037018301 (on the west side of Davidson). This seems to make sense, as it is more difficult to reach everybody at a similar time when there is a wider range of people. 

## 4: DEMOGRAPHIC CORRELATION ON THE INCIDENTS AND RESPONSE TIME DISTRIBUTION

- Get demographic information along with average response time for each tract into a CSV. Properly for the possibility that some values factored into averages may be null or less than zero.

In [ ]:
athena.start_query_execution(
    QueryString = 'SELECT geoid, wkt, COUNT(*) AS num_incidents, AVG(CASE WHEN median_household_income > 0 THEN median_household_income ELSE NULL END) AS median_household_income, AVG(CASE WHEN median_family_income > 0 THEN median_family_income ELSE NULL END) AS median_family_income, AVG(CASE WHEN median_housing_value > 0 THEN median_housing_value ELSE NULL END) AS median_housing_value, AVG(CASE WHEN median_gross_rent > 0 THEN median_gross_rent ELSE NULL END) AS median_gross_rent, AVG(CASE WHEN total_population > 0 THEN total_population ELSE NULL END) AS total_population, AVG(CASE WHEN white > 0 THEN white ELSE NULL END) AS white, AVG(CASE WHEN african_american > 0 THEN african_american ELSE NULL END) AS african_american, AVG(CASE WHEN hispanic > 0 THEN hispanic ELSE NULL END) AS hispanic, AVG(CASE WHEN response_time_sec > 0 THEN response_time_sec ELSE NULL END) AS avg_response_time FROM "econ_incidents" GROUP BY geoid, wkt;',
    QueryExecutionContext = {
        'Database': 'econ'
    },
    ResultConfiguration={
        'OutputLocation': 's3://final-proj-schachtj/output'
    }
)

{'QueryExecutionId': 'a3afcc66-3b7a-46d2-829c-5cfe1e6cd059',
 'ResponseMetadata': {'RequestId': '8507e569-feec-4ae0-a4b8-dfbe27dfe2a6',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 27 Apr 2023 21:05:54 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '59',
   'connection': 'keep-alive',
   'x-amzn-requestid': '8507e569-feec-4ae0-a4b8-dfbe27dfe2a6'},
  'RetryAttempts': 0}}

In [ ]:
athena.get_query_execution(QueryExecutionId = 'a3afcc66-3b7a-46d2-829c-5cfe1e6cd059')

{'QueryExecution': {'QueryExecutionId': 'a3afcc66-3b7a-46d2-829c-5cfe1e6cd059',
  'Query': 'SELECT geoid, wkt, COUNT(*) AS num_incidents, AVG(CASE WHEN median_household_income > 0 THEN median_household_income ELSE NULL END) AS median_household_income, AVG(CASE WHEN median_family_income > 0 THEN median_family_income ELSE NULL END) AS median_family_income, AVG(CASE WHEN median_housing_value > 0 THEN median_housing_value ELSE NULL END) AS median_housing_value, AVG(CASE WHEN median_gross_rent > 0 THEN median_gross_rent ELSE NULL END) AS median_gross_rent, AVG(CASE WHEN total_population > 0 THEN total_population ELSE NULL END) AS total_population, AVG(CASE WHEN white > 0 THEN white ELSE NULL END) AS white, AVG(CASE WHEN african_american > 0 THEN african_american ELSE NULL END) AS african_american, AVG(CASE WHEN hispanic > 0 THEN hispanic ELSE NULL END) AS hispanic, AVG(CASE WHEN response_time_sec > 0 THEN response_time_sec ELSE NULL END) AS avg_response_time FROM "econ_incidents" GROUP BY g

- Read into CSV

In [ ]:
s3.download_file('final-proj-schachtj', 'output/a3afcc66-3b7a-46d2-829c-5cfe1e6cd059.csv', 'avg_res_and_demographics.csv')

In [16]:
avg_res_and_demographics = pd.read_csv('avg_res_and_demographics.csv')
avg_res_and_demographics['geometry'] = avg_res_and_demographics['wkt'].apply(shapely.wkt.loads)
avg_res_and_demographics = avg_res_and_demographics.drop('wkt', axis=1)
avg_res_and_demographics

,geoid,num_incidents,median_household_income,median_family_income,median_housing_value,median_gross_rent,total_population,white,african_american,hispanic,avg_response_time,geometry
0,47037015620,203,51596.0,63377.0,137000.0,836.0,7713.0,3461.0,3204.0,1263.0,336.677596,"POLYGON ((-86.636183 36.071874, -86.635672 36...."
1,47037011600,26,51106.0,73015.0,163700.0,874.0,4071.0,2954.0,1090.0,NaN,392.200000,"POLYGON ((-86.727575 36.188247, -86.727374 36...."
2,47037010103,227,67250.0,80592.0,201500.0,985.0,2627.0,2422.0,144.0,39.0,639.928910,"POLYGON ((-86.917518 36.339757, -86.9174659999..."
3,47037015612,407,49878.0,59476.0,137200.0,981.0,6690.0,3552.0,2651.0,429.0,413.897098,"POLYGON ((-86.666478 36.103628, -86.666206 36...."
4,47037016000,200,34688.0,27262.0,112200.0,755.0,910.0,202.0,681.0,10.0,337.463158,"POLYGON ((-86.775159 36.149175, -86.775035 36...."
...,...,...,...,...,...,...,...,...,...,...,...,...
156,47037016700,144,94331.0,157991.0,606000.0,1382.0,5106.0,4597.0,150.0,92.0,375.201493,"POLYGON ((-86.847596 36.131537, -86.847411 36...."
157,47037013500,83,45625.0,56384.0,270000.0,875.0,2087.0,1402.0,503.0,130.0,374.986842,"POLYGON ((-86.836518 36.153923999999996, -86.8..."
158,47037012802,114,36855.0,43179.0,104700.0,928.0,4075.0,1136.0,2858.0,69.0,420.900901,"POLYGON ((-86.893164 36.211861, -86.893091 36...."
159,47037018407,34,95571.0,98209.0,248400.0,NaN,4520.0,3940.0,212.0,111.0,539.545455,"POLYGON ((-87.023179 36.076855, -87.022582 36...."


In [17]:
res_dem_gdf = gpd.GeoDataFrame(avg_res_and_demographics, crs='epsg:4326')
res_dem_gdf.index = res_dem_gdf['geoid']
res_dem_gdf

,geoid,num_incidents,median_household_income,median_family_income,median_housing_value,median_gross_rent,total_population,white,african_american,hispanic,avg_response_time,geometry
geoid,,,,,,,,,,,,
47037015620,47037015620,203,51596.0,63377.0,137000.0,836.0,7713.0,3461.0,3204.0,1263.0,336.677596,"POLYGON ((-86.63618 36.07187, -86.63567 36.073..."
47037011600,47037011600,26,51106.0,73015.0,163700.0,874.0,4071.0,2954.0,1090.0,NaN,392.200000,"POLYGON ((-86.72758 36.18825, -86.72737 36.188..."
47037010103,47037010103,227,67250.0,80592.0,201500.0,985.0,2627.0,2422.0,144.0,39.0,639.928910,"POLYGON ((-86.91752 36.33976, -86.91747 36.339..."
47037015612,47037015612,407,49878.0,59476.0,137200.0,981.0,6690.0,3552.0,2651.0,429.0,413.897098,"POLYGON ((-86.66648 36.10363, -86.66621 36.104..."
47037016000,47037016000,200,34688.0,27262.0,112200.0,755.0,910.0,202.0,681.0,10.0,337.463158,"POLYGON ((-86.77516 36.14917, -86.77504 36.149..."
...,...,...,...,...,...,...,...,...,...,...,...,...
47037016700,47037016700,144,94331.0,157991.0,606000.0,1382.0,5106.0,4597.0,150.0,92.0,375.201493,"POLYGON ((-86.84760 36.13154, -86.84741 36.132..."
47037013500,47037013500,83,45625.0,56384.0,270000.0,875.0,2087.0,1402.0,503.0,130.0,374.986842,"POLYGON ((-86.83652 36.15392, -86.83518 36.153..."
47037012802,47037012802,114,36855.0,43179.0,104700.0,928.0,4075.0,1136.0,2858.0,69.0,420.900901,"POLYGON ((-86.89316 36.21186, -86.89309 36.211..."


- See if there are any correlations between the racial makeup of a tract and the response times. This involves making scatter plots of response time versus racial makeup and comparing choropleth plots of racial makeup for each tract with the prior num_incidents and avg_response_time choropleths from Sections 2 and 3.

In [18]:
fig = px.scatter(res_dem_gdf, x='african_american', y='avg_response_time', trendline='ols',
    title="Response Time by African American Population for Census Tracts, Davidson County 2017-2021")
fig.show()
fig.write_html('res_time_by_african_american_pop.html')

In [19]:
fig = px.scatter(res_dem_gdf, x='white', y='avg_response_time', trendline="ols",
    title="Response Time by Average White Population for Census Tracts, Davidson County 2017-2021")
fig.show()
fig.write_html('res_time_by_white_pop.html')

In [20]:
fig = px.scatter(res_dem_gdf, x='hispanic', y='avg_response_time', trendline='ols',
                 title="Response Time by Average Hispanic Population for Census Tracts, Davidson County 2017-2021")
fig.show()
fig.write_html('res_time_by_hispanic_pop.html')

In [21]:
fig = px.choropleth(res_dem_gdf, geojson=res_dem_gdf, locations = res_dem_gdf.index,
                            color='african_american',
                            color_continuous_scale="Viridis",
                            range_color=(0, res_dem_gdf['african_american'].max()),
                            scope = "usa",
                            title="Number of African-American People by Census Tract, Davidson County 2017-2021")
fig.update_geos(fitbounds="locations", visible=True)
fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig.write_html('african_american_pop_by_tract.html')

In [22]:
fig = px.choropleth(res_dem_gdf, geojson=res_dem_gdf, locations = res_dem_gdf.index,
                            color='white',
                            color_continuous_scale="Viridis",
                            range_color=(0, res_dem_gdf['white'].max()),
                            scope = "usa",
                            title="Number of White People by Census Tract, Davidson County 2017-2021")
fig.update_geos(fitbounds="locations", visible=True)
fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig.write_html('white_pop_by_tract.html')

In [23]:
fig = px.choropleth(res_dem_gdf, geojson=res_dem_gdf, locations = res_dem_gdf.index,
                            color='hispanic',
                            color_continuous_scale="Viridis",
                            range_color=(0, res_dem_gdf['hispanic'].max()),
                            scope = "usa",
                            title="Number of Hispanic People by Census Tract, Davidson County 2017-2021")
fig.update_geos(fitbounds="locations", visible=True)
fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig.write_html('hispanic_pop_by_tract.html')

- Now make similar analyses based on the economic/financial makeup of a tract and the response times. This involves making scatter plots of response time versus various economic assessment factors and comparing choropleth plots of these economic assessment factors for each tract with the prior num_incidents and avg_response_time choropleths from Sections 2 and 3.

In [24]:
fig = px.scatter(res_dem_gdf, x='median_household_income', y='avg_response_time', trendline='ols',
    title="Response Time by Median Household Income for Census Tracts, Davidson County 2017-2021")
fig.show()
fig.write_html('response_time_by_median_household_income.html')

In [25]:
fig = px.scatter(res_dem_gdf, x='median_housing_value', y='avg_response_time', trendline='ols',
    title="Response Time by Median Housing Value for Census Tracts, Davidson County 2017-2021")
fig.show()
fig.write_html('response_time_by_median_housingval.html')

In [26]:
fig = px.choropleth(res_dem_gdf, geojson=res_dem_gdf, locations = res_dem_gdf.index,
                            color='median_household_income',
                            color_continuous_scale="Viridis",
                            range_color=(res_dem_gdf['median_household_income'].min(), res_dem_gdf['median_household_income'].max()),
                            scope = "usa",
                            title="Median Household Income by Census Tract, Davidson County 2017-2021")
fig.update_geos(fitbounds="locations", visible=True)
fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig.write_html('median_household_income_census_tract.html')

In [27]:
fig = px.choropleth(res_dem_gdf, geojson=res_dem_gdf, locations = res_dem_gdf.index,
                            color='median_housing_value',
                            color_continuous_scale="Viridis",
                            range_color=(res_dem_gdf['median_housing_value'].min(), res_dem_gdf['median_housing_value'].max()),
                            scope = "usa",
                            title="Median Housing Value by Census Tract, Davidson County 2017-2021")
fig.update_geos(fitbounds="locations", visible=True)
fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig.write_html('median_housing_val_census_tract.html')

- **Notable Conclusions for Section 4: Demographic Correlation on Incidents and Response Time Distribution**
    - There is a slightly positive correlation between the number of white people in a tract and the average response time for that region, which means it takes longer to reach the regions with higher white populations. This is explained by the choropleth of white population by tract, as the regions farther away from the city center generally have the highest white populations and the regions farther away from the city generally have longer response times (as shown by section 3). By contrast, the areas with the highest African-American and Hispanic populations are closer to the city center, which explains why the average response times for both demographics do not seem to change much as population increases. With that said, the number of incidents per capita is also significantly lower there. Any action that might be taken to increase emergency response presence in those outer regions might also impact this difference.
    - There is a slightly positive correlation between the income/housing prices in a tract and the average response time for that tract, which means it takes longer to reach the more affluent regions. These more affluent regions are generally located away from the city, which explains why they generally have longer response times. As with the point above, any action that might be taken to increase emergency response presence in those outer regions might impact this difference.

## FINAL CONCLUSIONS: SOCIOECONOMIC CENSUS DATA AND INCIDENTS
- Temporally, incidents occur most often during the winter months, so it is best to keep emergency response teams prepared during those times.
- Spatially, incidents are more common as you move closer to the city center, so it is good to have solid emergency response in the city. With that said, the average response time is better as you move closer to the city center. Some of the outer tracts with high response times have high populations (albeit low incident occurrence relative to the city center), namely the aforementioned tracts 47037019114 and 47037018301.
- Regions that are farther away from the city, which are also generally more affluent and have higher white populations, seem to have longer response times. With that said, they also have fewer emergency response incidents per capita. Heightening emergency response presence in regions farther away from the city may reduce response times for those areas, but it also may not be worth the money to establish them since those regions experience fewer incidents.

## SOURCES
- Geospatial queries: https://docs.aws.amazon.com/athena/latest/ug/geospatial-functions-list-v2.html, https://docs.aws.amazon.com/athena/latest/ug/geospatial-example-queries.html
- SQL joins: https://www.w3schools.com/sql/sql_join.asp
- Using wkt for CSV uploads to Athena: https://stackoverflow.com/questions/74179881/attributeerror-polygon-object-has-no-attribute-to-wkt
- Plotly documentation: https://plotly.com/python